In [ ]:
'''
3D UNET边缘检测。
'''

In [1]:
'''
统一设置地址
'''

import os

# 获取当前工作目录
current_dir = os.getcwd()
print("当前工作目录：", current_dir)

# 修改当前工作目录，以后输出文件只需要写文件名
new_dir = "D:/李娅宁/肩台外侧点-0715/"
os.chdir(new_dir)
print("修改后的工作目录：", os.getcwd())


当前工作目录： C:\Users\HP
修改后的工作目录： D:\李娅宁\肩台外侧点-0715


In [2]:
'''
数据预处理
'''

import os
import numpy as np
from scipy.interpolate import splprep, splev
import networkx as nx

def load_obj_file(file_path):
    vertices = []
    faces = []
    try:
        with open(file_path, 'r', encoding='utf-8') as file:
            for line in file:
                if line.startswith('v '):
                    parts = line.strip().split()
                    vertex = [float(parts[1]), float(parts[2]), float(parts[3])]
                    vertices.append(vertex)
                elif line.startswith('f '):
                    parts = line.strip().split()
                    face = [int(p.split('/')[0]) - 1 for p in parts[1:]]
                    faces.append(face)
    except FileNotFoundError:
        print(f"File not found: {file_path}")
    except Exception as e:
        print(f"An error occurred: {e}")
    return vertices, faces

def load_mark_file(file_path):
    marks = []
    try:
        with open(file_path, 'r', encoding='utf-8') as file:
            for line in file:
                parts = line.strip().split()
                if len(parts) == 3:
                    mark = [float(parts[0]), float(parts[1]), float(parts[2])]
                    marks.append(mark)
    except FileNotFoundError:
        print(f"File not found: {file_path}")
    except Exception as e:
        print(f"An error occurred: {e}")
    return marks

def center_vertices(vertices):
    vertices_array = np.array(vertices)
    min_coords = vertices_array.min(axis=0)
    max_coords = vertices_array.max(axis=0)
    center = (min_coords + max_coords) / 2
    centered_vertices = vertices_array - center
    return centered_vertices.tolist()

def interpolate_marks(marks, num_points=100):
    marks = np.array(marks)
    marks = np.vstack([marks, marks[0]])  # 添加第一个点到最后，使曲线闭合
    tck, u = splprep(marks.T, s=0, per=True)
    u_fine = np.linspace(0, 1, num_points)
    interpolated_marks = splev(u_fine, tck)
    return np.array(interpolated_marks).T

def generate_graph(vertices, faces):
    G = nx.Graph()
    for i, vertex in enumerate(vertices):
        G.add_node(i, pos=vertex)
    for face in faces:
        for i in range(len(face)):
            G.add_edge(face[i], face[(i+1) % len(face)])
    return G

def is_on_curve(vertex, curve_points, tolerance):
    for point in curve_points:
        if np.linalg.norm(vertex - point) < tolerance:
            return True
    return False

def segment_point_cloud(vertices, faces, interpolated_marks):
    tolerance = 0.1
    max_tolerance = 1.0
    tolerance_increment = 0.02

    while tolerance <= max_tolerance:
        G = generate_graph(vertices, faces)
        curve_points = np.array(interpolated_marks)
        
        # 找到封闭曲线上的顶点
        curve_points_indices = set()
        for i, vertex in enumerate(vertices):
            if is_on_curve(np.array(vertex), curve_points, tolerance):
                curve_points_indices.add(i)
        
        # 从图中删除封闭曲线上的点
        G.remove_nodes_from(curve_points_indices)
        
        # 使用图的连通性找到两个区域
        components = list(nx.connected_components(G))
        part1 = set()
        part2 = set()
        
        if len(components) > 1:
            part1 = components[0]
            part2 = components[1]
            # 返回所有点的标签：边缘点为 1，其余点为 0
            labels = [1 if i in curve_points_indices else 0 for i in range(len(vertices))]
            return part1, part2, labels, tolerance, len(curve_points_indices)
        else:
            tolerance += tolerance_increment
    
    return set(), set(), [0] * len(vertices), tolerance, len(curve_points_indices)

def save_points_with_labels_to_txt(vertices, labels, output_file_path):
    with open(output_file_path, 'w') as file:
        for i in range(len(vertices)):
            vertex = vertices[i]
            label = labels[i]
            file.write(f"{vertex[0]} {vertex[1]} {vertex[2]} {label}\n")

def process_files(input_folder, output_folder, threshold=200):
    for root, _, files in os.walk(input_folder):
        for file in files:
            if file.endswith('.obj'):
                obj_file_path = os.path.join(root, file)
                mark_file_path = obj_file_path + '.mark'
                if not os.path.exists(mark_file_path):
                    print(f"未找到mark文件: {mark_file_path}")
                    continue
                
                vertices, faces = load_obj_file(obj_file_path)
                marks = load_mark_file(mark_file_path)
                centered_vertices = center_vertices(vertices)
                interpolated_marks = interpolate_marks(marks)
                part1, part2, labels, tolerance, num_inside_points = segment_point_cloud(centered_vertices, faces, interpolated_marks)
                
                if len(part1) > 0 and len(part2) > 0 and num_inside_points < threshold:
                    print(f"{os.path.splitext(file)[0]} 的 tolerance是：{tolerance:.2f}, inside points 数目是：{num_inside_points}")
                    output_file_name = os.path.splitext(file)[0] + '_point_cloud_with_labels.txt'
                    output_file_path = os.path.join(output_folder, output_file_name)
                    save_points_with_labels_to_txt(centered_vertices, labels, output_file_path)
                else:
                    print(f"{os.path.splitext(file)[0]} 无法被分割为两部分")

# 使用修正后的文件路径
input_folder = r'肩台外侧点-0715'
output_folder = r'边缘检测预处理后数据'

if not os.path.exists(output_folder):
    os.makedirs(output_folder)

process_files(input_folder, output_folder)


1_1 的 tolerance是：0.18, inside points 数目是：130
1_10 的 tolerance是：0.18, inside points 数目是：119
1_2 的 tolerance是：0.16, inside points 数目是：105
1_3 的 tolerance是：0.20, inside points 数目是：142
1_4 的 tolerance是：0.18, inside points 数目是：125
1_5 的 tolerance是：0.20, inside points 数目是：133
未找到mark文件: 肩台外侧点-0715\1\1_6.obj.mark
1_7 的 tolerance是：0.16, inside points 数目是：118
1_8 的 tolerance是：0.18, inside points 数目是：117
1_9 的 tolerance是：0.16, inside points 数目是：104
10_1 的 tolerance是：0.20, inside points 数目是：132
10_10 的 tolerance是：0.18, inside points 数目是：113
10_2 的 tolerance是：0.20, inside points 数目是：129
10_3 的 tolerance是：0.20, inside points 数目是：129
10_4 的 tolerance是：0.18, inside points 数目是：123
10_5 的 tolerance是：0.18, inside points 数目是：108
10_6 的 tolerance是：0.18, inside points 数目是：108
10_7 的 tolerance是：0.18, inside points 数目是：120
10_8 的 tolerance是：0.20, inside points 数目是：129
10_9 的 tolerance是：0.18, inside points 数目是：117
101_1 的 tolerance是：0.32, inside points 数目是：197
101_10 无法被分割为两部分
101_2 的 tolerance是：0.22, inside 

126_4 的 tolerance是：0.24, inside points 数目是：154
126_5 的 tolerance是：0.22, inside points 数目是：144
126_6 无法被分割为两部分
126_7 无法被分割为两部分
126_8 无法被分割为两部分
126_9 的 tolerance是：0.24, inside points 数目是：162
127_1 的 tolerance是：0.20, inside points 数目是：118
127_10 的 tolerance是：0.20, inside points 数目是：126
127_2 的 tolerance是：0.22, inside points 数目是：152
127_3 的 tolerance是：0.22, inside points 数目是：138
127_4 的 tolerance是：0.22, inside points 数目是：151
127_5 的 tolerance是：0.22, inside points 数目是：154
127_6 的 tolerance是：0.26, inside points 数目是：182
127_7 的 tolerance是：0.24, inside points 数目是：170
127_8 无法被分割为两部分
127_9 的 tolerance是：0.24, inside points 数目是：160
128_1 无法被分割为两部分
128_10 的 tolerance是：0.30, inside points 数目是：198
128_2 无法被分割为两部分
128_3 的 tolerance是：0.22, inside points 数目是：133
128_4 的 tolerance是：0.20, inside points 数目是：121
128_5 无法被分割为两部分
128_6 的 tolerance是：0.26, inside points 数目是：163
128_7 无法被分割为两部分
128_8 的 tolerance是：0.24, inside points 数目是：155
128_9 的 tolerance是：0.20, inside points 数目是：117
129_1 的 tolerance是：0.14,

101 的 tolerance是：0.18, inside points 数目是：123
102 的 tolerance是：0.18, inside points 数目是：120
103 的 tolerance是：0.20, inside points 数目是：137
104 的 tolerance是：0.18, inside points 数目是：121
105 的 tolerance是：0.16, inside points 数目是：110
106 的 tolerance是：0.22, inside points 数目是：164
107 的 tolerance是：0.20, inside points 数目是：125
108 的 tolerance是：0.20, inside points 数目是：139
109 的 tolerance是：0.24, inside points 数目是：179
11 的 tolerance是：0.18, inside points 数目是：150
110 的 tolerance是：0.22, inside points 数目是：142
111 的 tolerance是：0.66, inside points 数目是：178
112 的 tolerance是：0.48, inside points 数目是：173
113 的 tolerance是：0.52, inside points 数目是：192
114 的 tolerance是：0.70, inside points 数目是：182
115 无法被分割为两部分
116 无法被分割为两部分
117 的 tolerance是：0.60, inside points 数目是：144
118 的 tolerance是：0.48, inside points 数目是：139
119 无法被分割为两部分
12 的 tolerance是：0.18, inside points 数目是：149
120 的 tolerance是：0.50, inside points 数目是：195
121 的 tolerance是：0.28, inside points 数目是：167
122 无法被分割为两部分
123 无法被分割为两部分
124 的 tolerance是：0.58, inside po

2_5 的 tolerance是：0.16, inside points 数目是：107
2_6 的 tolerance是：0.24, inside points 数目是：171
2_7 的 tolerance是：0.20, inside points 数目是：140
2_8 无法被分割为两部分
未找到mark文件: 肩台外侧点-0715\2\2_9.obj.mark
20_1 的 tolerance是：0.20, inside points 数目是：131
20_10 的 tolerance是：0.18, inside points 数目是：127
20_2 的 tolerance是：0.18, inside points 数目是：114
20_3 的 tolerance是：0.20, inside points 数目是：133
20_4 的 tolerance是：0.18, inside points 数目是：123
20_5 的 tolerance是：0.16, inside points 数目是：101
20_6 的 tolerance是：0.18, inside points 数目是：113
20_7 的 tolerance是：0.18, inside points 数目是：110
20_8 的 tolerance是：0.18, inside points 数目是：119
20_9 的 tolerance是：0.16, inside points 数目是：92
21_1 的 tolerance是：0.18, inside points 数目是：114
21_10 的 tolerance是：0.20, inside points 数目是：125
21_2 的 tolerance是：0.20, inside points 数目是：126
21_3 的 tolerance是：0.18, inside points 数目是：113
21_4 的 tolerance是：0.18, inside points 数目是：112
21_5 的 tolerance是：0.16, inside points 数目是：92
21_6 的 tolerance是：0.20, inside points 数目是：132
21_7 的 tolerance是：0.20, inside p

36_5 的 tolerance是：0.16, inside points 数目是：101
36_6 的 tolerance是：0.18, inside points 数目是：121
36_7 的 tolerance是：0.18, inside points 数目是：118
36_8 的 tolerance是：0.22, inside points 数目是：143
36_9 的 tolerance是：0.16, inside points 数目是：104
37_1 的 tolerance是：0.22, inside points 数目是：138
37_10 的 tolerance是：0.20, inside points 数目是：114
37_2 的 tolerance是：0.24, inside points 数目是：141
37_3 的 tolerance是：0.22, inside points 数目是：136
37_4 的 tolerance是：0.18, inside points 数目是：99
37_5 的 tolerance是：0.20, inside points 数目是：124
37_6 的 tolerance是：0.18, inside points 数目是：105
37_7 的 tolerance是：0.18, inside points 数目是：113
37_8 的 tolerance是：0.20, inside points 数目是：115
37_9 的 tolerance是：0.18, inside points 数目是：95
38_1 的 tolerance是：0.18, inside points 数目是：101
38_10 的 tolerance是：0.18, inside points 数目是：104
38_2 的 tolerance是：0.20, inside points 数目是：113
38_3 的 tolerance是：0.18, inside points 数目是：106
38_4 的 tolerance是：0.20, inside points 数目是：118
38_5 的 tolerance是：0.20, inside points 数目是：111
38_6 的 tolerance是：0.20, inside poi

56_5 的 tolerance是：0.18, inside points 数目是：116
56_6 的 tolerance是：0.26, inside points 数目是：186
56_7 的 tolerance是：0.22, inside points 数目是：159
56_8 的 tolerance是：0.20, inside points 数目是：124
56_9 无法被分割为两部分
57_1 的 tolerance是：0.20, inside points 数目是：118
57_10 的 tolerance是：0.22, inside points 数目是：136
57_2 的 tolerance是：0.22, inside points 数目是：140
57_3 的 tolerance是：0.20, inside points 数目是：140
57_4 的 tolerance是：0.24, inside points 数目是：164
57_5 的 tolerance是：0.24, inside points 数目是：159
57_6 的 tolerance是：0.24, inside points 数目是：151
57_7 的 tolerance是：0.24, inside points 数目是：162
57_8 的 tolerance是：0.20, inside points 数目是：117
57_9 的 tolerance是：0.20, inside points 数目是：119
58_1 的 tolerance是：0.18, inside points 数目是：112
58_10 的 tolerance是：0.24, inside points 数目是：137
58_2 的 tolerance是：0.22, inside points 数目是：141
58_3 的 tolerance是：0.22, inside points 数目是：130
58_4 的 tolerance是：0.22, inside points 数目是：130
58_5 的 tolerance是：0.26, inside points 数目是：157
58_6 的 tolerance是：0.24, inside points 数目是：138
58_7 的 tolerance是

In [2]:
'''
生成训练数据
'''

import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import os

class PointCloudDataset(Dataset):
    def __init__(self, txt_files, voxel_size=100):
        self.data = []
        for txt_file in txt_files:
            points = np.loadtxt(txt_file)
            self.data.append(self.voxelize(points, voxel_size))
    
    def voxelize(self, points, voxel_size):
        # Normalize points to fit in the voxel grid
        min_coords = points[:, :3].min(axis=0)
        max_coords = points[:, :3].max(axis=0)
        normalized_points = (points[:, :3] - min_coords) / (max_coords - min_coords)
        normalized_points *= (voxel_size - 1)
        
        # Create voxel grid
        voxel_grid = np.zeros((voxel_size, voxel_size, voxel_size), dtype=np.float32)
        for point in normalized_points:
            x, y, z = point.astype(int)
            voxel_grid[x, y, z] = 1  # Mark the voxel as occupied

        # Create label grid
        labels = np.zeros((voxel_size, voxel_size, voxel_size), dtype=np.float32)
        for point in points:
            x, y, z, label = point.astype(int)
            labels[x, y, z] = label  # Assign the label to the voxel

        return voxel_grid, labels
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        voxel_grid, labels = self.data[idx]
        return torch.tensor(voxel_grid).unsqueeze(0), torch.tensor(labels).unsqueeze(0)

def get_data_loaders(data_folder, batch_size=4, voxel_size=100):
    txt_files = [os.path.join(data_folder, f) for f in os.listdir(data_folder) if f.endswith('.txt')]
    dataset = PointCloudDataset(txt_files, voxel_size)
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
    return dataloader


OSError: [WinError 126] 找不到指定的模块。 Error loading "D:\Anaconda\Lib\site-packages\torch\lib\fbgemm.dll" or one of its dependencies.

In [ ]:
'''
3D UNET
'''

class UNet3D(nn.Module):
    def __init__(self, in_channels, out_channels, init_features=32):
        super(UNet3D, self).__init__()

        features = init_features
        self.encoder1 = UNet3D._block(in_channels, features)
        self.pool1 = nn.MaxPool3d(kernel_size=2, stride=2)
        self.encoder2 = UNet3D._block(features, features * 2)
        self.pool2 = nn.MaxPool3d(kernel_size=2, stride=2)
        self.encoder3 = UNet3D._block(features * 2, features * 4)
        self.pool3 = nn.MaxPool3d(kernel_size=2, stride=2)
        self.encoder4 = UNet3D._block(features * 4, features * 8)
        self.pool4 = nn.MaxPool3d(kernel_size=2, stride=2)

        self.bottleneck = UNet3D._block(features * 8, features * 16)

        self.upconv4 = nn.ConvTranspose3d(features * 16, features * 8, kernel_size=2, stride=2)
        self.decoder4 = UNet3D._block(features * 16, features * 8)
        self.upconv3 = nn.ConvTranspose3d(features * 8, features * 4, kernel_size=2, stride=2)
        self.decoder3 = UNet3D._block(features * 8, features * 4)
        self.upconv2 = nn.ConvTranspose3d(features * 4, features * 2, kernel_size=2, stride=2)
        self.decoder2 = UNet3D._block(features * 4, features * 2)
        self.upconv1 = nn.ConvTranspose3d(features * 2, features, kernel_size=2, stride=2)
        self.decoder1 = UNet3D._block(features * 2, features)

        self.conv = nn.Conv3d(in_channels=features, out_channels=out_channels, kernel_size=1)

    def forward(self, x):
        enc1 = self.encoder1(x)
        enc2 = self.encoder2(self.pool1(enc1))
        enc3 = self.encoder3(self.pool2(enc2))
        enc4 = self.encoder4(self.pool3(enc3))

        bottleneck = self.bottleneck(self.pool4(enc4))

        dec4 = self.upconv4(bottleneck)
        dec4 = torch.cat((dec4, enc4), dim=1)
        dec4 = self.decoder4(dec4)
        dec3 = self.upconv3(dec4)
        dec3 = torch.cat((dec3, enc3), dim=1)
        dec3 = self.decoder3(dec3)
        dec2 = self.upconv2(dec3)
        dec2 = torch.cat((dec2, enc2), dim=1)
        dec2 = self.decoder2(dec2)
        dec1 = self.upconv1(dec2)
        dec1 = torch.cat((dec1, enc1), dim=1)
        dec1 = self.decoder1(dec1)

        return torch.sigmoid(self.conv(dec1))

    @staticmethod
    def _block(in_channels, features, name=""):
        return nn.Sequential(
            nn.Conv3d(in_channels=in_channels, out_channels=features, kernel_size=3, padding=1, bias=False),
            nn.BatchNorm3d(features),
            nn.ReLU(inplace=True),
            nn.Conv3d(in_channels=features, out_channels=features, kernel_size=3, padding=1, bias=False),
            nn.BatchNorm3d(features),
            nn.ReLU(inplace=True),
        )



In [ ]:
'''
炼
'''

def dice_loss(pred, target, smooth=1e-6):
    intersection = (pred * target).sum()
    union = pred.sum() + target.sum()
    dice = (2.0 * intersection + smooth) / (union + smooth)
    return 1 - dice

def train(model, dataloader, optimizer, num_epochs=25, device='cuda'):
    model = model.to(device)
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        for inputs, labels in dataloader:
            inputs, labels = inputs.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(inputs)
            loss = dice_loss(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item() * inputs.size(0)

        epoch_loss = running_loss / len(dataloader.dataset)
        print(f'Epoch {epoch+1}/{num_epochs}, Loss: {epoch_loss:.4f}')

def evaluate(model, dataloader, device='cuda'):
    model.eval()
    total = 0
    correct = 0
    with torch.no_grad():
        for inputs, labels in dataloader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            predicted = (outputs > 0.5).float()
            total += labels.nelement()
            correct += (predicted == labels).sum().item()

    print(f'Accuracy: {correct / total:.4f}')

# 获取数据加载器
data_folder = r'边缘检测预处理后数据'
batch_size = 4
voxel_size = 100

dataloader = get_data_loaders(data_folder, batch_size, voxel_size)

# 定义模型、损失函数和优化器
model = UNet3D(in_channels=1, out_channels=1)
optimizer = optim.Adam(model.parameters(), lr=1e-4)

# 训练模型
train(model, dataloader, optimizer, num_epochs=2)

# 评估模型
evaluate(model, dataloader)

In [ ]:
# 保存模型
model_save_path = 'July29边缘检测模型/UNET_BEC.h5'
model.save(model_save_path)
print(f"Model saved to {model_save_path}")

In [ ]:
# 开启交互旋转
%matplotlib notebook


In [ ]:
'''
预测结果可视化
'''

import os
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import tensorflow as tf
from collections import defaultdict
from itertools import combinations

# 加载obj文件
def load_obj_file(file_path):
    vertices = []
    faces = []
    try:
        with open(file_path, 'r', encoding='utf-8') as file:
            for line in file:
                if line.startswith('v '):
                    parts = line.strip().split()
                    vertex = [float(parts[1]), float(parts[2]), float(parts[3])]
                    vertices.append(vertex)
                elif line.startswith('f '):
                    parts = line.strip().split()
                    face = [int(p.split('/')[0]) - 1 for p in parts[1:]]
                    faces.append(face)
    except FileNotFoundError:
        print(f"File not found: {file_path}")
    except Exception as e:
        print(f"An error occurred: {e}")
    return vertices, faces

# 将顶点平移至包围盒中心
def center_vertices(vertices):
    vertices_array = np.array(vertices)
    min_coords = vertices_array.min(axis=0)
    max_coords = vertices_array.max(axis=0)
    center = (min_coords + max_coords) / 2
    centered_vertices = vertices_array - center
    return centered_vertices.tolist(), center

# 点云转体素网格
def create_voxel_grid(data, grid_size):
    grid = np.zeros((grid_size, grid_size, grid_size))
    min_coords = np.min(data, axis=0)
    max_coords = np.max(data, axis=0)
    voxel_dim = (max_coords - min_coords) / grid_size

    for i, point in enumerate(data):
        voxel = ((point - min_coords) / voxel_dim).astype(int)
        voxel = np.clip(voxel, 0, grid_size-1)  # Ensure indices are within bounds
        grid[voxel[0], voxel[1], voxel[2]] = 1

    return grid, min_coords, voxel_dim

# 从训练好的模型获取标签
def get_labels_from_model(model, voxel_grid):
    voxel_grid = np.expand_dims(voxel_grid, axis=0)  # Add batch dimension
    voxel_grid = np.expand_dims(voxel_grid, axis=-1)  # Add channel dimension
    predictions = model.predict(voxel_grid)
    labels = (predictions > 0.5).astype(int)
    return labels.reshape(voxel_grid.shape[1], voxel_grid.shape[2], voxel_grid.shape[3])

# 应用预测标签到原始点云
def apply_labels_to_point_cloud(data, predicted_labels, min_coords, voxel_dim, grid_size):
    labels = np.zeros(len(data))
    for i, point in enumerate(data):
        voxel = ((point - min_coords) / voxel_dim).astype(int)
        voxel = np.clip(voxel, 0, grid_size-1)  # Ensure indices are within bounds
        labels[i] = predicted_labels[voxel[0], voxel[1], voxel[2]]
    return labels

# 绘制带有分类标签的点云和分界线
def plot_point_cloud_with_labels(vertices, labels, angles=(0, 0, 0)):
    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')

    vertices = np.array(vertices)
    labels = np.array(labels)

    # Apply rotation
    vertices = rotate_points(vertices, angles)
    
    x, y, z = vertices.T

    edge_points = vertices[labels == 1]
    non_edge_points = vertices[labels == 0]

    # 绘制点云
    ax.scatter(non_edge_points[:, 0], non_edge_points[:, 1], non_edge_points[:, 2], c='b', marker='o', s=1, label='Non-edge')
    ax.scatter(edge_points[:, 0], edge_points[:, 1], edge_points[:, 2], c='r', marker='o', s=1, label='Edge')

    # 设置标签和标题
    ax.set_xlabel('X axis')
    ax.set_ylabel('Y axis')
    ax.set_zlabel('Z axis')
    ax.set_title('3D Point Cloud with Edge Detection')

    # 确保坐标轴刻度一致
    max_range = np.array([max(x)-min(x), max(y)-min(y), max(z)-min(z)]).max()
    mid_x = (max(x) + min(x)) * 0.5
    mid_y = (max(y) + min(y)) * 0.5
    mid_z = (max(z) + min(z)) * 0.5
    ax.set_xlim(mid_x - max_range/2, mid_x + max_range/2)
    ax.set_ylim(mid_y - max_range/2, mid_y + max_range/2)
    ax.set_zlim(mid_z - max_range/2, mid_z + max_range/2)

    # 设置视角
    ax.view_init(elev=30, azim=30)  # Adjust these values as needed

    # 确保坐标轴比例相等
    ax.set_box_aspect([1,1,1])  # Aspect ratio is 1:1:1

    # 启用交互式旋转
    plt.legend()
    plt.show()

# 加载模型
model = tf.keras.models.load_model('July29边缘检测模型/UNET_BEC.h5')

# 加载点云数据
obj_file_path = r'肩台外侧点-0715/30/30_1.obj'
vertices, faces = load_obj_file(obj_file_path)
centered_vertices, center = center_vertices(vertices)

# 将点云转换为体素网格
grid_size = 100  # Grid size as specified
voxel_grid, min_coords, voxel_dim = create_voxel_grid(np.array(centered_vertices), grid_size)

# 使用训练好的模型进行预测
predicted_labels = get_labels_from_model(model, voxel_grid)

# 获取原始点云的预测标签
predicted_point_labels = apply_labels_to_point_cloud(np.array(centered_vertices), predicted_labels, min_coords, voxel_dim, grid_size)

# 画图
plot_point_cloud_with_labels(centered_vertices, predicted_point_labels, angles=(0, 0, 0))
